# Load data file

In [1]:
using DelimitedFiles

"""General Annealing Problem"""
abstract type AnnealingProblem end

"""
    SpinAnnealingProblem{T<:Real} <: AnnealingProblem

Annealing problem defined by coupling matrix of spins.
"""
struct SpinAnnealingProblem{T<:Real} <: AnnealingProblem  # immutable, with type parameter T (a subtype of Real).
    num_spin::Int
    coupling::Matrix{T}
    function SpinAnnealingProblem(coupling::Matrix{T}) where T
        size(coupling, 1) == size(coupling, 2) || throw(DimensionMismatch("input must be square matrix."))
        new{T}(size(coupling, 1), coupling)
    end
end

"""
    load_coupling(filename::String) -> SpinAnnealingProblem

Load the data file into symmtric coupling matrix.
"""
function load_coupling(filename::String)
    data = readdlm(filename)
    is = Int.(view(data, :, 1)) .+ 1
    js = Int.(view(data, :, 2)) .+ 1
    weights = data[:,3]
    num_spin = max(maximum(is), maximum(js))
    J = Matrix{eltype(weights)}(undef, num_spin, num_spin)
    for (i, j, weight) = zip(is, js, weights)
        J[i,j] = weight/2
        J[j,i] = weight/2
    end
    SpinAnnealingProblem(J)
end

load_coupling (generic function with 1 method)

In [2]:
@doc load_coupling

No documentation found.

`load_coupling` is a `Function`.

```
# 1 method for generic function "load_coupling":
[1] load_coupling(filename::String) in Main at In[1]:26
```


In [3]:
sap = load_coupling("example.txt")

SpinAnnealingProblem{Float64}(300, [0.0 0.5 … -0.5 -0.5; 0.5 0.0 … 0.5 -0.5; … ; -0.5 0.5 … 0.0 0.5; -0.5 -0.5 … 0.5 0.0])

In [65]:
abstract type AnnealingConfig end

struct SpinConfig{Ts, Tf} <: AnnealingConfig
    config::Vector{Ts}
    field::Vector{Tf}
    SpinConfig(config::Vector{Ts}, field::Vector{Tf}) where {Ts, Tf} = new{Ts, Tf}(config, field)
end

"""
    random_config(prblm::AnnealingProblem) -> SpinConfig

Random spin configuration.
"""
function random_config end

function random_config(prblm::SpinAnnealingProblem)
    config = rand([-1,1], prblm.num_spin)
    SpinConfig(config, prblm.coupling*config)
end

random_config (generic function with 1 method)

In [66]:
initial_config = random_config(sap)

SpinConfig{Int64,Float64}([1, 1, -1, 1, -1, -1, 1, -1, 1, -1  …  -1, -1, -1, 1, 1, 1, 1, 1, 1, 1], [-2.5, -9.5, 12.5, -4.5, 12.5, 9.5, -3.5, -5.5, 9.5, 2.5  …  4.5, -6.5, -12.5, -15.5, -3.5, -9.5, 0.5, 4.5, 2.5, -20.5])

In [76]:
"""
    anneal_singlerun!(config::AnnealingConfig, prblm, tempscales::Vector{Float64}, num_update_each_temp::Int)

Perform Simulated Annealing using Metropolis updates for the single run.

    * configuration that can be updated.
    * prblm: problem with `get_cost`, `flip!` and `random_config` interfaces.
    * tempscales: temperature scales, which should be a decreasing array.
    * num_update_each_temp: the number of update in each temprature scale.

Returns (minimum cost, optimal configuration).
"""
function anneal_singlerun!(config, prblm, tempscales::Vector{Float64}, num_update_each_temp::Int)
    cost = get_cost(config, prblm)
    
    opt_config = config
    opt_cost = cost
    for beta = 1 ./ tempscales
        @simd for m = 1:num_update_each_temp
            proposal, ΔE = propose(config, prblm)
            if exp(-beta*ΔE) > rand()  #accept
                flip!(config, proposal, prblm)
                cost += ΔE
                if cost < opt_cost
                    opt_cost = cost
                    opt_config = config
                end
            end
        end
    end
    opt_cost, opt_config
end
 
"""
    anneal(nrun::Int, prblm, tempscales::Vector{Float64}, num_update_each_temp::Int)

Perform Simulated Annealing with multiple runs.
"""
function anneal(nrun::Int, prblm, tempscales::Vector{Float64}, num_update_each_temp::Int)
    local opt_cost, opt_config
    for r = 1:nrun
        initial_config = random_config(prblm)
        cost, config = anneal_singlerun!(initial_config, prblm, tempscales, num_update_each_temp)
        if r == 1 || cost < opt_cost
            opt_cost = cost
            opt_config = config
        end
        # println("$r-th run, cost = $cost")
    end
    opt_cost, opt_config
end


anneal (generic function with 1 method)

In [77]:
get_cost(config::SpinConfig, sap::SpinAnnealingProblem) = sum(config.config'*sap.coupling*config.config)

@inline function propose(config::SpinConfig, ::SpinAnnealingProblem)  # ommit the name of argument, since not used.
    ispin = rand(1:length(config.config))
    @inbounds ΔE = -config.field[ispin] * config.config[ispin] * 4 # 2 for spin change, 2 for mutual energy.
    ispin, ΔE
end

@inline function flip!(config::SpinConfig, ispin::Int, sap::SpinAnnealingProblem)
    @inbounds config.config[ispin] = -config.config[ispin]
    # update field
    # config.field .+= 2 .* config.config[ispin] .* view(sap.coupling,:,ispin)
    @simd for i=1:sap.num_spin
        @inbounds config.field[i] += 2 * config.config[ispin] * sap.coupling[i,ispin]
    end
    config
    #* config.field .+= 2 .* config.config[ispin] .* sap.coupling[:,ispin]
end

flip! (generic function with 1 method)

In [78]:
tempscales = 10 .- (1:64 .- 1) .* 0.15 |> collect;

In [79]:
using BenchmarkTools
@benchmark anneal(30, sap, tempscales, 4000)

BenchmarkTools.Trial: 
  memory estimate:  398.00 KiB
  allocs estimate:  271
  --------------
  minimum time:     658.964 ms (0.00% GC)
  median time:      682.024 ms (0.00% GC)
  mean time:        704.124 ms (0.00% GC)
  maximum time:     846.126 ms (0.00% GC)
  --------------
  samples:          8
  evals/sample:     1

In [ ]:
# calling Fortran program
https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/index.html

In [ ]:
# @profile to see the bottleneck of this program
# @code_warntype to analyse the bottleneck.
